## Load latest headlines

In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-08-04-22-00 +0000,wsj,"Trump to Send 2,000 National Guard Troops to L...",https://www.wsj.com/us-news/california-protest...
1,2025-06-08-04-03-00 +0000,wsj,Israel Is Putting More Women on the Front Line...,https://www.wsj.com/world/middle-east/israel-i...
2,2025-06-08-04-02-00 +0000,wsj,Harvard’s China Ties Become New Front in Battl...,https://www.wsj.com/world/china/harvards-china...
3,2025-06-08-03-28-29 +0000,startribune,Teenager fatally shot Friday night outside Nor...,https://www.startribune.com/one-man-dead-in-sh...
4,2025-06-08-02-50-32 +0000,startribune,"What is the ‘House settlement,’ and what does ...",https://www.startribune.com/house-v-ncaa-settl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_27564\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,9
78,musk,6
77,elon,4
30,house,3
97,minnesota,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-06-07-09-00-00 +0000,wsj,Elon Musk and House Republicans both promised ...,https://www.wsj.com/politics/policy/republican...,39
26,2025-06-07-14-50-49 +0000,nypost,Elon Musk deletes tweet claiming Trump was ‘in...,https://nypost.com/2025/06/07/us-news/elon-mus...,33
19,2025-06-07-17-19-14 +0000,nypost,Trump threatens Elon Musk with ‘very serious c...,https://nypost.com/2025/06/07/us-news/trump-sa...,32
10,2025-06-08-00-53-31 +0000,nypost,Elon Musk body-checked Treasury Secretary Scot...,https://nypost.com/2025/06/07/us-news/elon-mus...,28
32,2025-06-07-10-23-07 +0000,nypost,Vance says Musk’s public feud with Trump is a ...,https://nypost.com/2025/06/07/us-news/jd-vance...,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,39,2025-06-07-09-00-00 +0000,wsj,Elon Musk and House Republicans both promised ...,https://www.wsj.com/politics/policy/republican...
18,22,2025-06-07-18-07-28 +0000,nypost,Trump warns of possible military action if Ira...,https://nypost.com/2025/06/07/us-news/trump-wa...
9,18,2025-06-08-01-00-00 +0000,wsj,Businesses are warning that constantly shiftin...,https://www.wsj.com/economy/trump-tariffs-us-e...
10,15,2025-06-08-00-53-31 +0000,nypost,Elon Musk body-checked Treasury Secretary Scot...,https://nypost.com/2025/06/07/us-news/elon-mus...
27,15,2025-06-07-14-39-12 +0000,nyt,Kilmar Abrego Garcia Returned to U.S. From El ...,https://www.nytimes.com/2025/06/06/us/politics...
24,14,2025-06-07-16-07-07 +0000,nypost,Bronx councilwoman’s bill would mandate life-s...,https://nypost.com/2025/06/07/us-news/new-bill...
29,13,2025-06-07-12-03-59 +0000,nypost,‘No show’ Delgado: NY’s lieutenant governor do...,https://nypost.com/2025/06/07/us-news/antonio-...
30,13,2025-06-07-12-00-00 +0000,startribune,Once-forgotten pits could be the most signific...,https://www.startribune.com/ancient-dakota-qua...
26,12,2025-06-07-14-50-49 +0000,nypost,Elon Musk deletes tweet claiming Trump was ‘in...,https://nypost.com/2025/06/07/us-news/elon-mus...
22,12,2025-06-07-16-59-12 +0000,nypost,"Hillary, Huma and Amal walk into a bar: Newest...",https://nypost.com/2025/06/07/us-news/the-newe...


## Next steps